In [1]:
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
games = pd.read_csv('steam.csv', nrows = 10000)

In [3]:
games['total_rating'] = games['positive_ratings'] - games['negative_ratings']


In [4]:
import pandas as pd


min_original_rating = games['total_rating'].min()
max_original_rating = games['total_rating'].max()

min_new_rating = 0
max_new_rating = 10

games['scaled_rating'] = ((games['total_rating'] - min_original_rating) / (max_original_rating - min_original_rating)) * (max_new_rating - min_new_rating) + min_new_rating


In [5]:
posters_df = pd.read_csv('steam_media_data.csv', nrows =10000) 


In [6]:
posters_df = posters_df.rename(columns={'steam_appid': 'appid'})

In [7]:
descriptions = pd.read_csv('steam_description_data.csv')
#description

## Data preparation

In [8]:
descriptions = descriptions.rename(columns={'steam_appid': 'appid'})
#descriptions

In [9]:
games = games.merge(descriptions, on = 'appid')


In [10]:
games = games.merge(posters_df, on = 'appid')
games.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,price,total_rating,scaled_rating,detailed_description,about_the_game,short_description,header_image,screenshots,background,movies
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,7.19,121195,0.557235,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,3.99,2685,0.029599,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,https://steamcdn-a.akamaihd.net/steam/apps/20/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,...,3.99,3018,0.031081,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,https://steamcdn-a.akamaihd.net/steam/apps/30/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/30/...,NaN
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,3.99,1006,0.022123,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,https://steamcdn-a.akamaihd.net/steam/apps/40/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,...,3.99,4962,0.039736,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,https://steamcdn-a.akamaihd.net/steam/apps/50/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN


In [11]:
# Replace 'selected_column' with the actual list of column names you want to select
selected_columns = ['appid','name', 'developer', 'categories', 'genres', 'steamspy_tags', 'price', 'detailed_description','header_image','scaled_rating']
games = games[selected_columns]
games.head(5)


,appid,name,developer,categories,genres,steamspy_tags,price,detailed_description,header_image,scaled_rating
0,10,Counter-Strike,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,7.19,Play the world's number 1 online action game. ...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,0.557235
1,20,Team Fortress Classic,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,3.99,One of the most popular online action games of...,https://steamcdn-a.akamaihd.net/steam/apps/20/...,0.029599
2,30,Day of Defeat,Valve,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,3.99,Enlist in an intense brand of Axis vs. Allied ...,https://steamcdn-a.akamaihd.net/steam/apps/30/...,0.031081
3,40,Deathmatch Classic,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,3.99,Enjoy fast-paced multiplayer gaming with Death...,https://steamcdn-a.akamaihd.net/steam/apps/40/...,0.022123
4,50,Half-Life: Opposing Force,Gearbox Software,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,3.99,Return to the Black Mesa Research Facility as ...,https://steamcdn-a.akamaihd.net/steam/apps/50/...,0.039736


In [12]:
highly_rated_games = games[games['scaled_rating'] > 0.25]
highly_rated_games['header_image'][0]


'https://steamcdn-a.akamaihd.net/steam/apps/10/header.jpg?t=1528733245'

### Data processing

In [13]:
#checking missing data
games.isnull().sum()

appid                   0
name                    0
developer               0
categories              0
genres                  0
steamspy_tags           0
price                   0
detailed_description    0
header_image            0
scaled_rating           0
dtype: int64

In [14]:
#checking duplicate data
games.duplicated().sum()

0

In [15]:
# Apply str.lower() to all columns with text data
games = games.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)
games

,appid,name,developer,categories,genres,steamspy_tags,price,detailed_description,header_image,scaled_rating
0,10,counter-strike,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,7.19,play the world's number 1 online action game. ...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,0.557235
1,20,team fortress classic,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,3.99,one of the most popular online action games of...,https://steamcdn-a.akamaihd.net/steam/apps/20/...,0.029599
2,30,day of defeat,valve,multi-player;valve anti-cheat enabled,action,fps;world war ii;multiplayer,3.99,enlist in an intense brand of axis vs. allied ...,https://steamcdn-a.akamaihd.net/steam/apps/30/...,0.031081
3,40,deathmatch classic,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,3.99,enjoy fast-paced multiplayer gaming with death...,https://steamcdn-a.akamaihd.net/steam/apps/40/...,0.022123
4,50,half-life: opposing force,gearbox software,single-player;multi-player;valve anti-cheat en...,action,fps;action;sci-fi,3.99,return to the black mesa research facility as ...,https://steamcdn-a.akamaihd.net/steam/apps/50/...,0.039736
...,...,...,...,...,...,...,...,...,...,...
9981,501350,dimension hunter vr,pocket money games,single-player,violent;gore;action;indie,action;indie;gore,4.79,"<img src=""https://steamcdn-a.akamaihd.net/stea...",https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017836
9982,501380,"uncorporeal - ""alcatraz island lofts""",uncorporeal systems,single-player,free to play;indie,free to play;indie;vr,0.00,"&quot;alcatraz island lofts&quot;, an experime...",https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017658
9983,501410,sos: special operative stories,attack mountain,single-player;steam trading cards,violent;action;adventure;indie,action;adventure;indie,4.99,"<img src=""https://steamcdn-a.akamaihd.net/stea...",https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017675
9984,501420,my own pet,jdmsoftware,single-player;steam achievements;steam cloud,casual;indie;simulation;early access,early access;indie;casual,4.79,&quot;my own pet&quot; is a game in which you ...,https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017644


In [16]:
# Function to preprocess and tokenize text
def preprocess_text(text):
   # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    return ' '.join(tokens)

# Apply preprocessing to the "detailed_description" column
games['detailed_description'] = games['detailed_description'].apply(preprocess_text)
games


,appid,name,developer,categories,genres,steamspy_tags,price,detailed_description,header_image,scaled_rating
0,10,counter-strike,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,7.19,play the worlds number 1 online action game en...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,0.557235
1,20,team fortress classic,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,3.99,one of the most popular online action games of...,https://steamcdn-a.akamaihd.net/steam/apps/20/...,0.029599
2,30,day of defeat,valve,multi-player;valve anti-cheat enabled,action,fps;world war ii;multiplayer,3.99,enlist in an intense brand of axis vs allied t...,https://steamcdn-a.akamaihd.net/steam/apps/30/...,0.031081
3,40,deathmatch classic,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,3.99,enjoy fastpaced multiplayer gaming with deathm...,https://steamcdn-a.akamaihd.net/steam/apps/40/...,0.022123
4,50,half-life: opposing force,gearbox software,single-player;multi-player;valve anti-cheat en...,action,fps;action;sci-fi,3.99,return to the black mesa research facility as ...,https://steamcdn-a.akamaihd.net/steam/apps/50/...,0.039736
...,...,...,...,...,...,...,...,...,...,...
9981,501350,dimension hunter vr,pocket money games,single-player,violent;gore;action;indie,action;indie;gore,4.79,img srchttpssteamcdnaakamaihdnetsteamapps50135...,https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017836
9982,501380,"uncorporeal - ""alcatraz island lofts""",uncorporeal systems,single-player,free to play;indie,free to play;indie;vr,0.00,quotalcatraz island loftsquot an experimental ...,https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017658
9983,501410,sos: special operative stories,attack mountain,single-player;steam trading cards,violent;action;adventure;indie,action;adventure;indie,4.99,img srchttpssteamcdnaakamaihdnetsteamapps50141...,https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017675
9984,501420,my own pet,jdmsoftware,single-player;steam achievements;steam cloud,casual;indie;simulation;early access,early access;indie;casual,4.79,quotmy own petquot is a game in which you will...,https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017644


In [17]:
columns_to_convert = ['name', 'developer', 'categories', 'genres','detailed_description']

# Loop through the specified columns and apply title case
for column in columns_to_convert:
    games[column] = games[column].str.title()
games


,appid,name,developer,categories,genres,steamspy_tags,price,detailed_description,header_image,scaled_rating
0,10,Counter-Strike,Valve,Multi-Player;Online Multi-Player;Local Multi-P...,Action,action;fps;multiplayer,7.19,Play The Worlds Number 1 Online Action Game En...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,0.557235
1,20,Team Fortress Classic,Valve,Multi-Player;Online Multi-Player;Local Multi-P...,Action,action;fps;multiplayer,3.99,One Of The Most Popular Online Action Games Of...,https://steamcdn-a.akamaihd.net/steam/apps/20/...,0.029599
2,30,Day Of Defeat,Valve,Multi-Player;Valve Anti-Cheat Enabled,Action,fps;world war ii;multiplayer,3.99,Enlist In An Intense Brand Of Axis Vs Allied T...,https://steamcdn-a.akamaihd.net/steam/apps/30/...,0.031081
3,40,Deathmatch Classic,Valve,Multi-Player;Online Multi-Player;Local Multi-P...,Action,action;fps;multiplayer,3.99,Enjoy Fastpaced Multiplayer Gaming With Deathm...,https://steamcdn-a.akamaihd.net/steam/apps/40/...,0.022123
4,50,Half-Life: Opposing Force,Gearbox Software,Single-Player;Multi-Player;Valve Anti-Cheat En...,Action,fps;action;sci-fi,3.99,Return To The Black Mesa Research Facility As ...,https://steamcdn-a.akamaihd.net/steam/apps/50/...,0.039736
...,...,...,...,...,...,...,...,...,...,...
9981,501350,Dimension Hunter Vr,Pocket Money Games,Single-Player,Violent;Gore;Action;Indie,action;indie;gore,4.79,Img Srchttpssteamcdnaakamaihdnetsteamapps50135...,https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017836
9982,501380,"Uncorporeal - ""Alcatraz Island Lofts""",Uncorporeal Systems,Single-Player,Free To Play;Indie,free to play;indie;vr,0.00,Quotalcatraz Island Loftsquot An Experimental ...,https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017658
9983,501410,Sos: Special Operative Stories,Attack Mountain,Single-Player;Steam Trading Cards,Violent;Action;Adventure;Indie,action;adventure;indie,4.99,Img Srchttpssteamcdnaakamaihdnetsteamapps50141...,https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017675
9984,501420,My Own Pet,Jdmsoftware,Single-Player;Steam Achievements;Steam Cloud,Casual;Indie;Simulation;Early Access,early access;indie;casual,4.79,Quotmy Own Petquot Is A Game In Which You Will...,https://steamcdn-a.akamaihd.net/steam/apps/501...,0.017644


In [18]:
# Combining relevant columns into a single column for TF-IDF vectorization
games['combined_features'] = games['categories'] + ' ' + games['genres'] + ' ' + games['developer']

In [19]:
# Creating a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the TF-IDF vectorizer on the combined features
tfidf_matrix = tfidf_vectorizer.fit_transform(games['combined_features'])


In [20]:
tfidf_matrix

<9986x7409 sparse matrix of type '<class 'numpy.float64'>'
	with 122204 stored elements in Compressed Sparse Row format>

In [21]:
# Compute the cosine similarity between games based on TF-IDF vectors
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [22]:
cosine_sim

array([[1.        , 1.        , 0.90872237, ..., 0.02534155, 0.01467719,
        0.0119792 ],
       [1.        , 1.        , 0.90872237, ..., 0.02534155, 0.01467719,
        0.0119792 ],
       [0.90872237, 0.90872237, 1.        , ..., 0.0199021 , 0.00569966,
        0.00465193],
       ...,
       [0.02534155, 0.02534155, 0.0199021 , ..., 1.        , 0.04611902,
        0.08535084],
       [0.01467719, 0.01467719, 0.00569966, ..., 0.04611902, 1.        ,
        0.08886418],
       [0.0119792 , 0.0119792 , 0.00465193, ..., 0.08535084, 0.08886418,
        1.        ]])

In [43]:
def get_game_recommendations(game_title, num_recommendations=10):
    # Get the index of the game in the dataset
    game_index = games.index[games['name'] == game_title].tolist()[0]

    # Get the cosine similarity scores for the game
    similar_scores = list(enumerate(cosine_sim[game_index]))

    # Sort the games by similarity score in descending order
    similar_games = sorted(similar_scores, key=lambda x: x[1], reverse=True)

    # Get the top N similar games
    top_similar_games = similar_games[1:num_recommendations + 1]
   

    # Extract the game titles
    recommended_games = list([games.iloc[i[0]]['name'],games.iloc[i[0]]['header_image']] for i in top_similar_games)
    return recommended_games

In [44]:
# Example usage:
user_searched_game = 'Counter-Strike'
recommendations = get_game_recommendations(user_searched_game)
if recommendations:
    print(f"Recommended games for '{user_searched_game}':")
    for game in recommendations:
        print(game)
else:
    print(f"Game '{user_searched_game}' not found in the dataset.")

Recommended games for 'Counter-Strike':
['Team Fortress Classic', 'https://steamcdn-a.akamaihd.net/steam/apps/20/header.jpg?t=1528732825']
['Deathmatch Classic', 'https://steamcdn-a.akamaihd.net/steam/apps/40/header.jpg?t=1528733362']
['Ricochet', 'https://steamcdn-a.akamaihd.net/steam/apps/60/header.jpg?t=1528733092']
['Half-Life', 'https://steamcdn-a.akamaihd.net/steam/apps/70/header.jpg?t=1530045175']
['Counter-Strike: Condition Zero', 'https://steamcdn-a.akamaihd.net/steam/apps/80/header.jpg?t=1512411962']
['Day Of Defeat', 'https://steamcdn-a.akamaihd.net/steam/apps/30/header.jpg?t=1512413490']
['Half-Life Deathmatch: Source', 'https://steamcdn-a.akamaihd.net/steam/apps/360/header.jpg?t=1530046445']
['Half-Life 2: Deathmatch', 'https://steamcdn-a.akamaihd.net/steam/apps/320/header.jpg?t=1512752170']
['Lost Planet™: Extreme Condition', 'https://steamcdn-a.akamaihd.net/steam/apps/6510/header.jpg?t=1543619216']
['Call Of Duty: Black Ops - Mac Edition', 'https://steamcdn-a.akamaihd.ne

In [25]:
import pickle

In [26]:
pickle.dump(games.to_dict(),open('games_dict.pkl','wb'))

In [27]:
games['name'].values

array(['Counter-Strike', 'Team Fortress Classic', 'Day Of Defeat', ...,
       'Sos: Special Operative Stories', 'My Own Pet',
       'Blue Solar: Chaos War'], dtype=object)

In [28]:
games.to_dict()

{'appid': {0: 10,
  1: 20,
  2: 30,
  3: 40,
  4: 50,
  5: 60,
  6: 70,
  7: 80,
  8: 130,
  9: 220,
  10: 240,
  11: 280,
  12: 300,
  13: 320,
  14: 340,
  15: 360,
  16: 380,
  17: 400,
  18: 420,
  19: 440,
  20: 500,
  21: 550,
  22: 570,
  23: 620,
  24: 630,
  25: 730,
  26: 1002,
  27: 1200,
  28: 1250,
  29: 1300,
  30: 1500,
  31: 1510,
  32: 1520,
  33: 1530,
  34: 1600,
  35: 1610,
  36: 1630,
  37: 1640,
  38: 1670,
  39: 1690,
  40: 1700,
  41: 1840,
  42: 1900,
  43: 1930,
  44: 2100,
  45: 2200,
  46: 2210,
  47: 2270,
  48: 2280,
  49: 2290,
  50: 2300,
  51: 2310,
  52: 2320,
  53: 2330,
  54: 2340,
  55: 2350,
  56: 2360,
  57: 2370,
  58: 2390,
  59: 2400,
  60: 2450,
  61: 2500,
  62: 2520,
  63: 2590,
  64: 2600,
  65: 2610,
  66: 2620,
  67: 2630,
  68: 2640,
  69: 2710,
  70: 2720,
  71: 2780,
  72: 2800,
  73: 2810,
  74: 2820,
  75: 2840,
  76: 2850,
  77: 2870,
  78: 2900,
  79: 2910,
  80: 2920,
  81: 2990,
  82: 3010,
  83: 3020,
  84: 3050,
  85: 3130,
  8

In [29]:
pickle.dump(cosine_sim,open('ifidf_matrix.pkl','wb'))

In [30]:
pickle.dump(games,open('games.pkl','wb'))

In [31]:
pickle.dump(highly_rated_games,open('highly_rated_games.pkl','wb'))